In [1]:
# Load necessary packages
library(ggplot2)
library(psych)
library(dplyr)
library(fmsb)
library(ggradar)
library(plotrix)


Attaching package: ‘psych’


The following objects are masked from ‘package:ggplot2’:

    %+%, alpha



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘plotrix’


The following object is masked from ‘package:psych’:

    rescale




In [2]:
# radarchart function to plot data
# altered dfmax to be 1.25x larger than the max value, making the figures more readable
radarchart <- function(df, axistype=0, seg=4, pty=16, pcol=1:8, plty=1:6, plwd=1,
                       pdensity=NULL, pangle=45, pfcol=NA, cglty=3, cglwd=1,
                       cglcol="navy", axislabcol="blue", title="", maxmin=TRUE,
                       na.itp=TRUE, centerzero=FALSE, vlabels=NULL, vlcex=NULL,
                       caxislabels=NULL, calcex=NULL,
                       paxislabels=NULL, palcex=NULL, ...) {
  if (!is.data.frame(df)) { cat("The data must be given as dataframe.\n"); return() }
  if ((n <- length(df))<3) { cat("The number of variables must be 3 or more.\n"); return() }
  if (maxmin==FALSE) { # when the dataframe does not include max and min as the top 2 rows.
    dfmax <- apply(df, 2, max)
    dfmax <- round(dfmax+(dfmax/4), digits=2)
    dfmin <- apply(df, 2, min)
    df <- rbind(dfmax, dfmin, df)
  }
    # Initialize variables to store the coordinates of the first two series
  first_series <- NULL
  second_series <- NULL
    
  plot(c(-1.2, 1.2), c(-1.2, 1.2), type="n", frame.plot=FALSE, axes=FALSE, 
       xlab="", ylab="", main=title, asp=1, ...) # define x-y coordinates without any plot
  theta <- seq(90, 450, length=n+1)*pi/180
  theta <- theta[1:n]
  xx <- cos(theta)
  yy <- sin(theta)
  CGap <- ifelse(centerzero, 0, 1)
  for (i in 0:seg) { # complementary guide lines, dotted navy line by default
    polygon(xx*(i+CGap)/(seg+CGap), yy*(i+CGap)/(seg+CGap), lty=cglty, lwd=cglwd, border=cglcol)
    if (axistype==1|axistype==3) CAXISLABELS <- paste(i/seg*100,"(%)")
    if (axistype==4|axistype==5) CAXISLABELS <- sprintf("%3.2f",i/seg)
    if (!is.null(caxislabels)&(i<length(caxislabels))) CAXISLABELS <- caxislabels[i+1]
    if (axistype==1|axistype==3|axistype==4|axistype==5) {
     if (is.null(calcex)) text(-0.05, (i+CGap)/(seg+CGap), CAXISLABELS, col=axislabcol) else
     text(-0.05, (i+CGap)/(seg+CGap), CAXISLABELS, col=axislabcol, cex=calcex)
    }
  }
  if (centerzero) {
    arrows(0, 0, xx*1, yy*1, lwd=cglwd, lty=cglty, length=0, col=cglcol)
  }
  else {
    arrows(xx/(seg+CGap), yy/(seg+CGap), xx*1, yy*1, lwd=cglwd, lty=cglty, length=0, col=cglcol)
  }
  PAXISLABELS <- df[1,1:n]
  if (!is.null(paxislabels)) PAXISLABELS <- paxislabels
  if (axistype==2|axistype==3|axistype==5) {
   if (is.null(palcex)) text(xx[1:n], yy[1:n], PAXISLABELS, col=axislabcol) else
   text(xx[1:n], yy[1:n], PAXISLABELS, col=axislabcol, cex=palcex)
  }
  VLABELS <- colnames(df)
  if (!is.null(vlabels)) VLABELS <- vlabels
  if (is.null(vlcex)) text(xx*1.2, yy*1.2, VLABELS) else
  text(xx*1.2, yy*1.2, VLABELS, cex=vlcex)
  series <- length(df[[1]])
  SX <- series-2
  if (length(pty) < SX) { ptys <- rep(pty, SX) } else { ptys <- pty }
  if (length(pcol) < SX) { pcols <- rep(pcol, SX) } else { pcols <- pcol }
  if (length(plty) < SX) { pltys <- rep(plty, SX) } else { pltys <- plty }
  if (length(plwd) < SX) { plwds <- rep(plwd, SX) } else { plwds <- plwd }
  if (length(pdensity) < SX) { pdensities <- rep(pdensity, SX) } else { pdensities <- pdensity }
  if (length(pangle) < SX) { pangles <- rep(pangle, SX)} else { pangles <- pangle }
  if (length(pfcol) < SX) { pfcols <- rep(pfcol, SX) } else { pfcols <- pfcol }
  for (i in 3:series) {
    xxs <- xx
    yys <- yy
    scale <- CGap/(seg+CGap)+(df[i,]-df[2,])/(df[1,]-df[2,])*seg/(seg+CGap)
    if (sum(!is.na(df[i,]))<3) { cat(sprintf("[DATA NOT ENOUGH] at %d\n%g\n",i,df[i,])) # for too many NA's (1.2.2012)
    } else {
      for (j in 1:n) {
        if (is.na(df[i, j])) { # how to treat NA
          if (na.itp) { # treat NA using interpolation
            left <- ifelse(j>1, j-1, n)
            while (is.na(df[i, left])) {
              left <- ifelse(left>1, left-1, n)
            }
            right <- ifelse(j<n, j+1, 1)
            while (is.na(df[i, right])) {
              right <- ifelse(right<n, right+1, 1)
            }
            xxleft <- xx[left]*CGap/(seg+CGap)+xx[left]*(df[i,left]-df[2,left])/(df[1,left]-df[2,left])*seg/(seg+CGap)
            yyleft <- yy[left]*CGap/(seg+CGap)+yy[left]*(df[i,left]-df[2,left])/(df[1,left]-df[2,left])*seg/(seg+CGap)
            xxright <- xx[right]*CGap/(seg+CGap)+xx[right]*(df[i,right]-df[2,right])/(df[1,right]-df[2,right])*seg/(seg+CGap)
            yyright <- yy[right]*CGap/(seg+CGap)+yy[right]*(df[i,right]-df[2,right])/(df[1,right]-df[2,right])*seg/(seg+CGap)
            if (xxleft > xxright) {
              xxtmp <- xxleft; yytmp <- yyleft;
              xxleft <- xxright; yyleft <- yyright;
              xxright <- xxtmp; yyright <- yytmp;
            }
            xxs[j] <- xx[j]*(yyleft*xxright-yyright*xxleft)/(yy[j]*(xxright-xxleft)-xx[j]*(yyright-yyleft))
            yys[j] <- (yy[j]/xx[j])*xxs[j]
          } else { # treat NA as zero (origin)
            xxs[j] <- 0
            yys[j] <- 0
          }
        }
        else {
          xxs[j] <- xx[j]*CGap/(seg+CGap)+xx[j]*(df[i, j]-df[2, j])/(df[1, j]-df[2, j])*seg/(seg+CGap)
          yys[j] <- yy[j]*CGap/(seg+CGap)+yy[j]*(df[i, j]-df[2, j])/(df[1, j]-df[2, j])*seg/(seg+CGap)
        }
      }
      if (is.null(pdensities)) {
        polygon(xxs, yys, lty=pltys[i-2], lwd=plwds[i-2], border=pcols[i-2], col=pfcols[i-2])
      } else {
        polygon(xxs, yys, lty=pltys[i-2], lwd=plwds[i-2], border=pcols[i-2], 
         density=pdensities[i-2], angle=pangles[i-2], col=pfcols[i-2])
      }
      points(xx*scale, yy*scale, pch=ptys[i-2], col=pcols[i-2])
    }
            # Store the coordinates of the first two series
    if (i == 3) {
      first_series <- list(x = xxs, y = yys)
    } else if (i == 4) {
      second_series <- list(x = xxs, y = yys)
    }
  }
    # Add a shaded area between the first two series
    if (!is.null(first_series) && !is.null(second_series)) {
        # Correct closing points for each series
        xxs <- c(first_series$x, first_series$x[1])  # Close the first series at its first point
        yys <- c(first_series$y, first_series$y[1])
        xxs <- c(xxs, rev(second_series$x), second_series$x[length(second_series$x)])  # Close the polygon with the last point of the second series
        yys <- c(yys, rev(second_series$y), second_series$y[length(second_series$y)])
        polygon(xxs, yys, col = rgb(0.5, 0.5, 0.5, 0.2), border = NA)
    }
}

In [3]:
# Read patient data
patient_data <- read.csv("/home/mtweed/scratch/tractoflow_hcp_dwi/patient_data.csv")

# Read DTI metric data for caudate, hippocampus, nucleus accumbens, pallidum, and putamen
ad_data <- read.csv("~/linear_regression/ad_other_rois.csv")
fa_data <- read.csv("~/linear_regression/fa_other_rois.csv")
md_data <- read.csv("~/linear_regression/md_other_rois.csv")
rd_data <- read.csv("~/linear_regression/rd_other_rois.csv")

# Read DTI metric data for hypothalamus
HCP_ad_data <- read.csv("~/linear_regression//HCP_ad_data_bin.csv")
HCP_fa_data <- read.csv("~/linear_regression//HCP_fa_data_bin.csv")
HCP_md_data <- read.csv("~/linear_regression//HCP_md_data_bin.csv")
HCP_rd_data <- read.csv("~/linear_regression//HCP_rd_data_bin.csv")

In [4]:
# Identify all non-twins
pd_nt=patient_data[patient_data$ZygositySR=='NotTwin',]

# Identify all twins and remove one of them
pd_t=patient_data[(patient_data$ZygositySR=='MZ' | patient_data$ZygositySR=='NotMZ'),]
pd_t=pd_t[!duplicated(pd_t$Family_ID),]
patient_data=rbind(pd_nt,pd_t)

In [5]:
# Filter rows of HCP DTI metric data based on matching subjects in patient_data
ad_data <- ad_data[ad_data$Subject %in% patient_data$Subject, ]
fa_data <- fa_data[fa_data$Subject %in% patient_data$Subject, ]
md_data <- md_data[md_data$Subject %in% patient_data$Subject, ]
rd_data <- rd_data[rd_data$Subject %in% patient_data$Subject, ]

# Filter rows of HCP DTI metric data based on matching subjects in patient_data
HCP_ad_data <- HCP_ad_data[HCP_ad_data$Subject %in% patient_data$Subject, ]
HCP_fa_data <- HCP_fa_data[HCP_fa_data$Subject %in% patient_data$Subject, ]
HCP_md_data <- HCP_md_data[HCP_md_data$Subject %in% patient_data$Subject, ]
HCP_rd_data <- HCP_rd_data[HCP_rd_data$Subject %in% patient_data$Subject, ]

In [6]:
# Define hypothalamus ROI names
ad_hypo <- c("ad_anterior_hypothalamic_area", "ad_arcuate_hypothalamic_nucleus", "ad_dorsomedial_hypothalamic_nucleus", "ad_lateral_hypothalamus", "ad_mammillary_bodies", "ad_paraventricular_nucleus", "ad_periventricular_hypothalamic_nucleus", "ad_posterior_hypothalamic_nucleus", "ad_suprachiasmatic_hypothalamic_nucleus", "ad_supraoptic_hypothalamic_nucleus", "ad_tuberomammillary_hypothalamic_nucleus", "ad_ventromedial_hypothalamus")
fa_hypo <- c("fa_anterior_hypothalamic_area", "fa_arcuate_hypothalamic_nucleus", "fa_dorsal_periventricular_hypothalamic_nucleus", "fa_dorsomedial_hypothalamic_nucleus", "fa_lateral_hypothalamus", "fa_mammillary_bodies", "fa_medial_preoptic_nucleus", "fa_paraventricular_nucleus", "fa_periventricular_hypothalamic_nucleus", "fa_posterior_hypothalamic_nucleus", "fa_suprachiasmatic_hypothalamic_nucleus", "fa_supraoptic_hypothalamic_nucleus", "fa_tuberomammillary_hypothalamic_nucleus", "fa_ventromedial_hypothalamus")
md_hypo <- c("md_anterior_hypothalamic_area", "md_arcuate_hypothalamic_nucleus", "md_dorsal_periventricular_hypothalamic_nucleus", "md_dorsomedial_hypothalamic_nucleus", "md_lateral_hypothalamus", "md_mammillary_bodies", "md_medial_preoptic_nucleus", "md_paraventricular_nucleus", "md_periventricular_hypothalamic_nucleus", "md_posterior_hypothalamic_nucleus", "md_suprachiasmatic_hypothalamic_nucleus", "md_supraoptic_hypothalamic_nucleus", "md_tuberomammillary_hypothalamic_nucleus", "md_ventromedial_hypothalamus")
rd_hypo <- c("rd_anterior_hypothalamic_area", "rd_arcuate_hypothalamic_nucleus", "rd_dorsal_periventricular_hypothalamic_nucleus", "rd_dorsomedial_hypothalamic_nucleus", "rd_lateral_hypothalamus", "rd_mammillary_bodies", "rd_medial_preoptic_nucleus", "rd_paraventricular_nucleus", "rd_periventricular_hypothalamic_nucleus", "rd_posterior_hypothalamic_nucleus", "rd_suprachiasmatic_hypothalamic_nucleus", "rd_supraoptic_hypothalamic_nucleus", "rd_tuberomammillary_hypothalamic_nucleus", "rd_ventromedial_hypothalamus")

# Calculate the value of the full hypothalmus for each DTI metric and add to table with other brain regions
ad_data$full_hypo <- rowMeans(HCP_ad_data[, ad_hypo], na.rm = TRUE)
fa_data$full_hypo <- rowMeans(HCP_fa_data[, fa_hypo], na.rm = TRUE)
md_data$full_hypo <- rowMeans(HCP_md_data[, md_hypo], na.rm = TRUE)
rd_data$full_hypo <- rowMeans(HCP_rd_data[, rd_hypo], na.rm = TRUE)

In [7]:
# Remove outliers for AD data
Q3_c=as.numeric(quantile(ad_data$caudate_spm,0.75, na.rm=TRUE))
Q1_c=as.numeric(quantile(ad_data$caudate_spm,0.25, na.rm=TRUE))
upper_c=Q3_c+(2.2*(Q3_c-Q1_c))
lower_c=Q1_c-(2.2*(Q3_c-Q1_c))
Q3_pu=as.numeric(quantile(ad_data$putamen_spm,0.75, na.rm=TRUE))
Q1_pu=as.numeric(quantile(ad_data$putamen_spm,0.25, na.rm=TRUE))
upper_pu=Q3_pu+(2.2*(Q3_pu-Q1_pu))
lower_pu=Q1_pu-(2.2*(Q3_pu-Q1_pu))
Q3_n=as.numeric(quantile(ad_data$nacc_spm,0.75, na.rm=TRUE))
Q1_n=as.numeric(quantile(ad_data$nacc_spm,0.25, na.rm=TRUE))
upper_n=Q3_n+(2.2*(Q3_n-Q1_n))
lower_n=Q1_n-(2.2*(Q3_n-Q1_n))
Q3_h=as.numeric(quantile(ad_data$hippocampus_spm,0.75, na.rm=TRUE))
Q1_h=as.numeric(quantile(ad_data$hippocampus_spm,0.25, na.rm=TRUE))
upper_h=Q3_h+(2.2*(Q3_h-Q1_h))
lower_h=Q1_h-(2.2*(Q3_h-Q1_h))
Q3_pa=as.numeric(quantile(ad_data$pallidum_spm,0.75, na.rm=TRUE))
Q1_pa=as.numeric(quantile(ad_data$pallidum_spm,0.25, na.rm=TRUE))
upper_pa=Q3_pa+(2.2*(Q3_pa-Q1_pa))
lower_pa=Q1_pa-(2.2*(Q3_pa-Q1_pa))
Q3_hy=as.numeric(quantile(ad_data$full_hypo,0.75, na.rm=TRUE))
Q1_hy=as.numeric(quantile(ad_data$full_hypo,0.25, na.rm=TRUE))
upper_hy=Q3_hy+(2.2*(Q3_hy-Q1_hy))
lower_hy=Q1_hy-(2.2*(Q3_hy-Q1_hy))
ad_data$caudate_spm[ad_data$caudate_spm<lower_c]=NA
ad_data$caudate_spm[ad_data$caudate_spm>upper_c]=NA
ad_data$putamen_spm[ad_data$putamen_spm<lower_pu]=NA
ad_data$putamen_spm[ad_data$putamen_spm>upper_pu]=NA
ad_data$nacc_spm[ad_data$nacc_spm<lower_n]=NA
ad_data$nacc_spm[ad_data$nacc_spm>upper_n]=NA
ad_data$hippocampus_spm[ad_data$hippocampus_spm<lower_h]=NA
ad_data$hippocampus_spm[ad_data$hippocampus_spm>upper_h]=NA
ad_data$pallidum_spm[ad_data$pallidum_spm<lower_pa]=NA
ad_data$pallidum_spm[ad_data$pallidum_spm>upper_pa]=NA
ad_data$full_hypo[ad_data$full_hypo<lower_hy]=NA
ad_data$full_hypo[ad_data$full_hypo>upper_hy]=NA

# Remove outliers for FA data
Q3_c=as.numeric(quantile(fa_data$caudate_spm,0.75, na.rm=TRUE))
Q1_c=as.numeric(quantile(fa_data$caudate_spm,0.25, na.rm=TRUE))
upper_c=Q3_c+(2.2*(Q3_c-Q1_c))
lower_c=Q1_c-(2.2*(Q3_c-Q1_c))
Q3_pu=as.numeric(quantile(fa_data$putamen_spm,0.75, na.rm=TRUE))
Q1_pu=as.numeric(quantile(fa_data$putamen_spm,0.25, na.rm=TRUE))
upper_pu=Q3_pu+(2.2*(Q3_pu-Q1_pu))
lower_pu=Q1_pu-(2.2*(Q3_pu-Q1_pu))
Q3_n=as.numeric(quantile(fa_data$nacc_spm,0.75, na.rm=TRUE))
Q1_n=as.numeric(quantile(fa_data$nacc_spm,0.25, na.rm=TRUE))
upper_n=Q3_n+(2.2*(Q3_n-Q1_n))
lower_n=Q1_n-(2.2*(Q3_n-Q1_n))
Q3_h=as.numeric(quantile(fa_data$hippocampus_spm,0.75, na.rm=TRUE))
Q1_h=as.numeric(quantile(fa_data$hippocampus_spm,0.25, na.rm=TRUE))
upper_h=Q3_h+(2.2*(Q3_h-Q1_h))
lower_h=Q1_h-(2.2*(Q3_h-Q1_h))
Q3_pa=as.numeric(quantile(fa_data$pallidum_spm,0.75, na.rm=TRUE))
Q1_pa=as.numeric(quantile(fa_data$pallidum_spm,0.25, na.rm=TRUE))
upper_pa=Q3_pa+(2.2*(Q3_pa-Q1_pa))
lower_pa=Q1_pa-(2.2*(Q3_pa-Q1_pa))
Q3_hy=as.numeric(quantile(fa_data$full_hypo,0.75, na.rm=TRUE))
Q1_hy=as.numeric(quantile(fa_data$full_hypo,0.25, na.rm=TRUE))
upper_hy=Q3_hy+(2.2*(Q3_hy-Q1_hy))
lower_hy=Q1_hy-(2.2*(Q3_hy-Q1_hy))
fa_data$caudate_spm[fa_data$caudate_spm<lower_c]=NA
fa_data$caudate_spm[fa_data$caudate_spm>upper_c]=NA
fa_data$putamen_spm[fa_data$putamen_spm<lower_pu]=NA
fa_data$putamen_spm[fa_data$putamen_spm>upper_pu]=NA
fa_data$nacc_spm[fa_data$nacc_spm<lower_n]=NA
fa_data$nacc_spm[fa_data$nacc_spm>upper_n]=NA
fa_data$hippocampus_spm[fa_data$hippocampus_spm<lower_h]=NA
fa_data$hippocampus_spm[fa_data$hippocampus_spm>upper_h]=NA
fa_data$pallidum_spm[fa_data$pallidum_spm<lower_pa]=NA
fa_data$pallidum_spm[fa_data$pallidum_spm>upper_pa]=NA
fa_data$full_hypo[fa_data$full_hypo<lower_hy]=NA
fa_data$full_hypo[fa_data$full_hypo>upper_hy]=NA

# Remove outliers for MD data
Q3_c=as.numeric(quantile(md_data$caudate_spm,0.75, na.rm=TRUE))
Q1_c=as.numeric(quantile(md_data$caudate_spm,0.25, na.rm=TRUE))
upper_c=Q3_c+(2.2*(Q3_c-Q1_c))
lower_c=Q1_c-(2.2*(Q3_c-Q1_c))
Q3_pu=as.numeric(quantile(md_data$putamen_spm,0.75, na.rm=TRUE))
Q1_pu=as.numeric(quantile(md_data$putamen_spm,0.25, na.rm=TRUE))
upper_pu=Q3_pu+(2.2*(Q3_pu-Q1_pu))
lower_pu=Q1_pu-(2.2*(Q3_pu-Q1_pu))
Q3_n=as.numeric(quantile(md_data$nacc_spm,0.75, na.rm=TRUE))
Q1_n=as.numeric(quantile(md_data$nacc_spm,0.25, na.rm=TRUE))
upper_n=Q3_n+(2.2*(Q3_n-Q1_n))
lower_n=Q1_n-(2.2*(Q3_n-Q1_n))
Q3_h=as.numeric(quantile(md_data$hippocampus_spm,0.75, na.rm=TRUE))
Q1_h=as.numeric(quantile(md_data$hippocampus_spm,0.25, na.rm=TRUE))
upper_h=Q3_h+(2.2*(Q3_h-Q1_h))
lower_h=Q1_h-(2.2*(Q3_h-Q1_h))
Q3_pa=as.numeric(quantile(md_data$pallidum_spm,0.75, na.rm=TRUE))
Q1_pa=as.numeric(quantile(md_data$pallidum_spm,0.25, na.rm=TRUE))
upper_pa=Q3_pa+(2.2*(Q3_pa-Q1_pa))
lower_pa=Q1_pa-(2.2*(Q3_pa-Q1_pa))
Q3_hy=as.numeric(quantile(md_data$full_hypo,0.75, na.rm=TRUE))
Q1_hy=as.numeric(quantile(md_data$full_hypo,0.25, na.rm=TRUE))
upper_hy=Q3_hy+(2.2*(Q3_hy-Q1_hy))
lower_hy=Q1_hy-(2.2*(Q3_hy-Q1_hy))
md_data$caudate_spm[md_data$caudate_spm<lower_c]=NA
md_data$caudate_spm[md_data$caudate_spm>upper_c]=NA
md_data$putamen_spm[md_data$putamen_spm<lower_pu]=NA
md_data$putamen_spm[md_data$putamen_spm>upper_pu]=NA
md_data$nacc_spm[md_data$nacc_spm<lower_n]=NA
md_data$nacc_spm[md_data$nacc_spm>upper_n]=NA
md_data$hippocampus_spm[md_data$hippocampus_spm<lower_h]=NA
md_data$hippocampus_spm[md_data$hippocampus_spm>upper_h]=NA
md_data$pallidum_spm[md_data$pallidum_spm<lower_pa]=NA
md_data$pallidum_spm[md_data$pallidum_spm>upper_pa]=NA
md_data$full_hypo[md_data$full_hypo<lower_hy]=NA
md_data$full_hypo[md_data$full_hypo>upper_hy]=NA

# Remove outliers for RD data
Q3_c=as.numeric(quantile(rd_data$caudate_spm,0.75, na.rm=TRUE))
Q1_c=as.numeric(quantile(rd_data$caudate_spm,0.25, na.rm=TRUE))
upper_c=Q3_c+(2.2*(Q3_c-Q1_c))
lower_c=Q1_c-(2.2*(Q3_c-Q1_c))
Q3_pu=as.numeric(quantile(rd_data$putamen_spm,0.75, na.rm=TRUE))
Q1_pu=as.numeric(quantile(rd_data$putamen_spm,0.25, na.rm=TRUE))
upper_pu=Q3_pu+(2.2*(Q3_pu-Q1_pu))
lower_pu=Q1_pu-(2.2*(Q3_pu-Q1_pu))
Q3_n=as.numeric(quantile(rd_data$nacc_spm,0.75, na.rm=TRUE))
Q1_n=as.numeric(quantile(rd_data$nacc_spm,0.25, na.rm=TRUE))
upper_n=Q3_n+(2.2*(Q3_n-Q1_n))
lower_n=Q1_n-(2.2*(Q3_n-Q1_n))
Q3_h=as.numeric(quantile(rd_data$hippocampus_spm,0.75, na.rm=TRUE))
Q1_h=as.numeric(quantile(rd_data$hippocampus_spm,0.25, na.rm=TRUE))
upper_h=Q3_h+(2.2*(Q3_h-Q1_h))
lower_h=Q1_h-(2.2*(Q3_h-Q1_h))
Q3_pa=as.numeric(quantile(rd_data$pallidum_spm,0.75, na.rm=TRUE))
Q1_pa=as.numeric(quantile(rd_data$pallidum_spm,0.25, na.rm=TRUE))
upper_pa=Q3_pa+(2.2*(Q3_pa-Q1_pa))
lower_pa=Q1_pa-(2.2*(Q3_pa-Q1_pa))
Q3_hy=as.numeric(quantile(rd_data$full_hypo,0.75, na.rm=TRUE))
Q1_hy=as.numeric(quantile(rd_data$full_hypo,0.25, na.rm=TRUE))
upper_hy=Q3_hy+(2.2*(Q3_hy-Q1_hy))
lower_hy=Q1_hy-(2.2*(Q3_hy-Q1_hy))
rd_data$caudate_spm[rd_data$caudate_spm<lower_c]=NA
rd_data$caudate_spm[rd_data$caudate_spm>upper_c]=NA
rd_data$putamen_spm[rd_data$putamen_spm<lower_pu]=NA
rd_data$putamen_spm[rd_data$putamen_spm>upper_pu]=NA
rd_data$nacc_spm[rd_data$nacc_spm<lower_n]=NA
rd_data$nacc_spm[rd_data$nacc_spm>upper_n]=NA
rd_data$hippocampus_spm[rd_data$hippocampus_spm<lower_h]=NA
rd_data$hippocampus_spm[rd_data$hippocampus_spm>upper_h]=NA
rd_data$pallidum_spm[rd_data$pallidum_spm<lower_pa]=NA
rd_data$pallidum_spm[rd_data$pallidum_spm>upper_pa]=NA
rd_data$full_hypo[rd_data$full_hypo<lower_hy]=NA
rd_data$full_hypo[rd_data$full_hypo>upper_hy]=NA

In [8]:
ad_data$HbA1C[ad_data$HbA1C == 1.32] <- NA
ad_data$HbA1C[ad_data$HbA1C == 3.12] <- NA
ad_data$HbA1C[ad_data$HbA1C == 3.26] <- NA
ad_data$HbA1C[ad_data$HbA1C == 9.6] <- NA

fa_data$HbA1C[fa_data$HbA1C == 1.32] <- NA
fa_data$HbA1C[fa_data$HbA1C == 3.12] <- NA
fa_data$HbA1C[fa_data$HbA1C == 3.26] <- NA
fa_data$HbA1C[fa_data$HbA1C == 9.6] <- NA

md_data$HbA1C[md_data$HbA1C == 1.32] <- NA
md_data$HbA1C[md_data$HbA1C == 3.12] <- NA
md_data$HbA1C[md_data$HbA1C == 3.26] <- NA
md_data$HbA1C[md_data$HbA1C == 9.6] <- NA

rd_data$HbA1C[rd_data$HbA1C == 1.32] <- NA
rd_data$HbA1C[rd_data$HbA1C == 3.12] <- NA
rd_data$HbA1C[rd_data$HbA1C == 3.26] <- NA
rd_data$HbA1C[rd_data$HbA1C == 9.6] <- NA

In [9]:
write.csv(ad_data, "~/linear_regression/HCP_AD_SUBJ_FINAL")
write.csv(fa_data, "~/linear_regression/HCP_FA_SUBJ_FINAL")
write.csv(md_data, "~/linear_regression/HCP_MD_SUBJ_FINAL")
write.csv(rd_data, "~/linear_regression/HCP_RD_SUBJ_FINAL")

In [10]:
# Name and structure necessary lists and data frames to fit linear models
subject_data <- list(ad_data, fa_data, md_data, rd_data)
subject_datasets <- c("ad", "fa", "md", "rd")
brain_regions <- c("caudate_spm", "putamen_spm", "nacc_spm", "hippocampus_spm", "pallidum_spm", "full_hypo")
brain <- c("caudate", "putamen", "nacc", "hippocampus", "pallidum", "full_hypo")
variables <- c("BMI", "BPSystolic", "BPDiastolic", "HbA1C")
vars <- c("BMI", "SBP", "DBP", "HbA1c")
all_data_summary <- data.frame()

In [11]:
# Loop through subject_data and fit linear regression models for each combination of brain region and variable
for (i in seq_along(subject_data)) {
    current_data <- subject_data[[i]]
    j=1
    for (region in brain_regions) {
        k=1
        for (variable in variables) {
            # Fit linear regression
            lm_model <- lm(get(region) ~ get(variable) + 
                           Age_in_Yrs + 
                           poly(Age_in_Yrs, 2, raw=TRUE) * Gender, data = current_data)
            # Summarize the model and extract coefficients
            summary_model <- summary(lm_model)
            values <- data.frame(summary_model$coefficients[2,])
            values <- as.data.frame(t(values))
            # Add patient MRI data to the results
            values$metric <- subject_datasets[i]
            values$region <- region
            values$measure <- vars[k]
            # Rename and append results to the overall summary dataset
            row_name <- paste(subject_datasets[i],"_", region, "_", variable)
            rownames(values) <- row_name 
            all_data_summary <- rbind(all_data_summary, values)
            k=k+1
        j=j+1
        }
    }
}
# Write data summary as a CSV
write.csv(all_data_summary, "~/linear_regression/HCP_all_data_summary")

In [12]:
# Define variables with linear regression data for each brain region
all_data_summary <- read.csv("~/linear_regression/HCP_all_data_summary")
caudate_data_summary <- all_data_summary[all_data_summary$region == 'caudate_spm',]
putamen_data_summary <- all_data_summary[all_data_summary$region == 'putamen_spm',] 
nacc_data_summary <- all_data_summary[all_data_summary$region == 'nacc_spm',] 
hippocampus_data_summary <- all_data_summary[all_data_summary$region == 'hippocampus_spm',] 
pallidum_data_summary <- all_data_summary[all_data_summary$region == 'pallidum_spm',] 
full_hypo_data_summary <- all_data_summary[all_data_summary$region == 'full_hypo',] 

# Calculate P_FDR for each brain region to account for multiple comparisons
caudate_data_summary$p_fdr = p.adjust(caudate_data_summary$Pr...t.., method="fdr")
putamen_data_summary$p_fdr = p.adjust(putamen_data_summary$Pr...t.., method="fdr")
nacc_data_summary$p_fdr = p.adjust(nacc_data_summary$Pr...t.., method="fdr")
hippocampus_data_summary$p_fdr = p.adjust(hippocampus_data_summary$Pr...t.., method="fdr")
pallidum_data_summary$p_fdr = p.adjust(pallidum_data_summary$Pr...t.., method="fdr")
full_hypo_data_summary$p_fdr = p.adjust(full_hypo_data_summary$Pr...t.., method="fdr")

# Recombine data of all brain regions into one data frame
all_data_summary <- data.frame()
all_data_summary <- bind_rows(caudate_data_summary, putamen_data_summary)
all_data_summary <- bind_rows(all_data_summary, nacc_data_summary)
all_data_summary <- bind_rows(all_data_summary, hippocampus_data_summary)
all_data_summary <- bind_rows(all_data_summary, pallidum_data_summary)
all_data_summary <- bind_rows(all_data_summary, full_hypo_data_summary)

In [13]:
# Define brain regions in two variables, one to match the CSV and other to name in the radar chart 
brain_regions <- c("caudate", "putamen", "nacc", "hippocampus", "pallidum", "full_hypo")
regions <- c("Caudate", "Putamen", "Nucleus Accumbens", "Hippocampus", "Pallidum", "Hypothalamus")

# Extract brain region based on variable measured
BMI_data <- all_data_summary[all_data_summary$measure == 'BMI',] 
hba1c_data <- all_data_summary[all_data_summary$measure == 'HbA1c',] 
sys_bp_data <- all_data_summary[all_data_summary$measure == 'SBP',] 
dia_bp_data <- all_data_summary[all_data_summary$measure == 'DBP',]

In [14]:
# Extract linear regression data for caudate data
ad_caudate_data <- all_data_summary[all_data_summary$metric == 'ad' & all_data_summary$region == 'caudate_spm',] 
fa_caudate_data <- all_data_summary[all_data_summary$metric == 'fa' & all_data_summary$region == 'caudate_spm',] 
md_caudate_data <- all_data_summary[all_data_summary$metric == 'md' & all_data_summary$region == 'caudate_spm',] 
rd_caudate_data <- all_data_summary[all_data_summary$metric == 'rd' & all_data_summary$region == 'caudate_spm',] 

# Extract linear regression data for putamen data
ad_putamen_data <- all_data_summary[all_data_summary$metric == 'ad' & all_data_summary$region == 'putamen_spm',] 
fa_putamen_data <- all_data_summary[all_data_summary$metric == 'fa' & all_data_summary$region == 'putamen_spm',] 
md_putamen_data <- all_data_summary[all_data_summary$metric == 'md' & all_data_summary$region == 'putamen_spm',] 
rd_putamen_data <- all_data_summary[all_data_summary$metric == 'rd' & all_data_summary$region == 'putamen_spm',]

# Extract linear regression data for nacc data
ad_nacc_data <- all_data_summary[all_data_summary$metric == 'ad' & all_data_summary$region == 'nacc_spm',] 
fa_nacc_data <- all_data_summary[all_data_summary$metric == 'fa' & all_data_summary$region == 'nacc_spm',] 
md_nacc_data <- all_data_summary[all_data_summary$metric == 'md' & all_data_summary$region == 'nacc_spm',] 
rd_nacc_data <- all_data_summary[all_data_summary$metric == 'rd' & all_data_summary$region == 'nacc_spm',]

# Extract linear regression data for hippocampus data
ad_hippocampus_data <- all_data_summary[all_data_summary$metric == 'ad' & all_data_summary$region == 'hippocampus_spm',] 
fa_hippocampus_data <- all_data_summary[all_data_summary$metric == 'fa' & all_data_summary$region == 'hippocampus_spm',] 
md_hippocampus_data <- all_data_summary[all_data_summary$metric == 'md' & all_data_summary$region == 'hippocampus_spm',] 
rd_hippocampus_data <- all_data_summary[all_data_summary$metric == 'rd' & all_data_summary$region == 'hippocampus_spm',]

# Extract linear regression data for pallidum data
ad_pallidum_data <- all_data_summary[all_data_summary$metric == 'ad' & all_data_summary$region == 'pallidum_spm',] 
fa_pallidum_data <- all_data_summary[all_data_summary$metric == 'fa' & all_data_summary$region == 'pallidum_spm',] 
md_pallidum_data <- all_data_summary[all_data_summary$metric == 'md' & all_data_summary$region == 'pallidum_spm',] 
rd_pallidum_data <- all_data_summary[all_data_summary$metric == 'rd' & all_data_summary$region == 'pallidum_spm',]

# Extract linear regression data for hypothalamus data
ad_full_hypo_data <- all_data_summary[all_data_summary$metric == 'ad' & all_data_summary$region == 'full_hypo',] 
fa_full_hypo_data <- all_data_summary[all_data_summary$metric == 'fa' & all_data_summary$region == 'full_hypo',] 
md_full_hypo_data <- all_data_summary[all_data_summary$metric == 'md' & all_data_summary$region == 'full_hypo',] 
rd_full_hypo_data <- all_data_summary[all_data_summary$metric == 'rd' & all_data_summary$region == 'full_hypo',]

In [18]:
# Define lists and variables for radar charts
vars <- c("BMI", "SBP", "DBP", "HbA1c")
region_data <- list()
region_data_summary <- list()
spider_plots <- list()

# Open TIFF file
tiff('~/scratch/tractoflow_hcp_dwi/spider_plots/spider_plots_hcp.tiff', width=8000, height=5000, res=300)

# Set plot parameters
par(mar=c(3,3,3,3))
par(mfrow = c(2, 3))

# Loop through brain regions
for (i in 1:length(brain_regions)) {
    # Subset data based on FDR significance
    significant_data <- subset(get(paste(brain_regions[i],"_data_summary",sep="")), p_fdr> 0.05)
    # Calculate min and max t-values
    min_t_val<- min(significant_data$t.value, na.rm = TRUE)
    max_t_val<- max(significant_data$t.value, na.rm = TRUE)
    # Create data frames for min and max t-values
    t_min <- data.frame(t(data.frame(replicate(length(vars), min_t_val))))
    colnames(t_min) <- vars
    t_max <- data.frame(t(data.frame(replicate(length(vars), max_t_val))))
    colnames(t_max) <- vars
    
    # Create a data frame for max and min values
    max_t <- max(get(paste(brain_regions[i],"_data_summary",sep=""))$t.value)
    min_t <- min(get(paste(brain_regions[i],"_data_summary",sep=""))$t.value)
    max_min <- data.frame(
    BMI = c(max_t, min_t),
    SBP = c(max_t, min_t), 
    DBP = c(max_t, min_t),
    HbA1c = c(max_t, min_t))
    
    # Create data frames for region-specific t-values
    region_data[[1]] <- data.frame(t(get(paste("ad_",brain_regions[i],"_data",sep=""))$t.value))
    colnames(region_data[[1]]) = vars
    region_data[[2]] <- data.frame(t(get(paste("fa_",brain_regions[i],"_data",sep=""))$t.value))
    colnames(region_data[[2]]) = vars
    region_data[[3]] <- data.frame(t(get(paste("md_",brain_regions[i],"_data",sep=""))$t.value))
    colnames(region_data[[3]]) = vars
    region_data[[4]] <- data.frame(t(get(paste("rd_",brain_regions[i],"_data",sep=""))$t.value))
    colnames(region_data[[4]]) = vars

    # Combine region-specific t-values into a summary data frame
    region_data_summary[[i]] <- bind_rows(max_min, region_data[[1]])
    region_data_summary[[i]] <- bind_rows(region_data_summary[i], region_data[[2]])
    region_data_summary[[i]] <- bind_rows(region_data_summary[i], region_data[[3]])
    region_data_summary[[i]] <- bind_rows(region_data_summary[i], region_data[[4]])
    region_data_summary[[i]] <- bind_rows(region_data_summary[i], t_min)
    region_data_summary[[i]] <- bind_rows(region_data_summary[i], t_max)
    rownames(region_data_summary[[i]]) = c("Max", "Min", "AD", "FA", "MD", "RD", "Min_T", "Max_T")
    data <- region_data_summary[[i]][c("Max", "Min", "Max_T", "Min_T", "AD", "FA", "MD", "RD"), ]

    # Define colors to be used in radar charts
    my_colors <- c( "black", "black","#ED64C9", "#FAA46A", "#944BE3", "#62C0FE")

    # Generate radar charts
    spider_plots[[i]] <- radarchart(
      data,  
      axistype = 1, 
      caxislabels = round(seq(min_t, max_t, ((max_t - min_t) / 4)), 1),
      title = paste(regions[i]),
      cex.main=3,
      pcol = my_colors,
      pfcol = c(NA,NA,NA, NA, NA, NA),
      pty = c(32 ,32 ,16, 16, 16, 16),
      plwd = c(2,2,4,4,4,4),  
      plty = c(1,1,3,3,3,3),
      cglcol = "grey", 
      cglty = 1,  
      cglwd = 0.8,
      axislabcol='black',
      seg=4,
      calcex=2.5,
      vlcex=2.8
    )
    
    # Add legend
    legend(
      "bottom", 
      legend = rownames(data[-c(1, 2, 3, 4), ]),
      fill = my_colors[3:6],
      bty = "n",
      ncol = length(rownames(data[-c(1, 2, 3, 4), ])),  
      cex = 2.5,
      inset = c(0, -0.06), 
      xpd = TRUE  
    )
}

# Close TIFF file
dev.off()

png 
  2

In [17]:
# Write data summary as a CSV
write.csv(all_data_summary, "/home/mtweed/scratch/tractoflow_hcp_dwi/spider_plots/HCP_data_final.csv")